In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

## With the keywords loaded and exceptions

In [2]:
# import pandas as pd
# import re
# from tqdm import tqdm
#
# # --- CONFIG ---
# INPUT_FILE = "olx_land_sale.csv"
# OUTPUT_FILE = "dataset_with_price_text1.csv"
# TEXT_COLUMN = "description_"
# CONTEXT_WORDS = 3  # Number of words before and after
#
# def extract_price_with_context(text):
#     """
#     Extract price per land unit with 3 words before and after for context.
#     Returns None if no valid price pattern found.
#     """
#     if not isinstance(text, str) or text.strip() == "":
#         return None
#
#     # Clean up HTML tags but preserve text
#     text = re.sub(r'<br\s*/?>', ' ', text)
#     text = re.sub(r'<[^>]+>', '', text)
#     text = ' '.join(text.split())  # Normalize whitespace
#
#     # Patterns here:
#     patterns = [
#         #1: "22 000 за сотку" / "15000 за сотку"
#         r'(\d{3,}[\s,]*\d*)\s*(?:у\.е\.|usd|\$|доллар|dollar|ming|минг)?\s*(за|per|for)\s*(сотку|sotku|sotix|sot|сотих|сот|гектар|gektar)',
#
#         # 2: "сотиги 5500$" / "sotixi 20000"
#         r'(сотих[иы]|sotix[iy]|сот[иы]г[иы])\s*(\d{3,}[\s,]*\d*)\s*(?:\$|у\.е\.|usd|ming|минг|dollar)?',
#
#         # 3: "1 сотих 80.000 u.e" / "1 sotix 50000$"
#         r'(1\s*(?:сотку|сотих|сотка|sotix|sotku|sot))\s*(\d{3,}[\s,\.]*\d*)\s*(?:\$|у\.е\.|usd|ming|минг)?',
#
#         # 4: "цена 22 000 за сотку"
#         r'(цена|нарх[иы]?|price|cost)\s*:?\s*(\d{3,}[\s,]*\d*)\s*(?:\$|у\.е\.|usd|ming|минг)?\s*(за|per)\s*(сотку|sotix|сот)',
#
#         # 5: "5000 мингдан" (Uzbek "from price")
#         r'(\d{3,}[\s,]*\d*)\s*(ming|минг)(дан)',
#
#         # 6: "сотку: 15000" or "за сотку 20000"
#         r'(за|per)?\s*(сотку|сотих|sotix|sotku|сот)\s*[:-]?\s*(\d{4,}[\s,]*\d*)\s*(?:\$|у\.е\.|ming|минг|usd)?',
#     ]
#
#     best_match = None
#     best_match_pos = -1
#
#     for pattern in patterns:
#         matches = list(re.finditer(pattern, text, re.IGNORECASE))
#         for match in matches:
#             # Get the full matched text
#             matched_text = match.group(0)
#
#             # Filter out matches that are too short or just numbers
#             if len(matched_text.strip()) < 5:
#                 continue
#
#             # Check if this match contains actual price indicators
#             has_price_word = any(word in matched_text.lower() for word in
#                 ['сотку', 'sotix', 'сот', 'за', 'ming', 'минг', 'цена', 'нарх'])
#
#             if not has_price_word:
#                 continue
#
#             # Extract numbers and verify they're realistic prices
#             numbers = re.findall(r'\d+', matched_text.replace(' ', '').replace(',', ''))
#             if numbers:
#                 main_number = max(numbers, key=lambda x: len(x))
#                 # Filter unrealistic prices (too small)
#                 if len(main_number) < 3:
#                     continue
#
#             # Get context: 3 words before and after
#             start_pos = match.start()
#             end_pos = match.end()
#
#             # Find word boundaries before match
#             before_text = text[:start_pos]
#             before_words = before_text.split()[-CONTEXT_WORDS:]
#
#             # Find word boundaries after match
#             after_text = text[end_pos:]
#             after_words = after_text.split()[:CONTEXT_WORDS]
#
#             # Build context string
#             context = ' '.join(before_words + [matched_text] + after_words)
#
#             # Keep the first good match found (or prioritize by position)
#             if best_match is None or start_pos < best_match_pos:
#                 best_match = context.strip()
#                 best_match_pos = start_pos
#
#     return best_match if best_match else None
#
# # --- LOAD DATA ---
# print(" Loading data...")
# df = pd.read_csv(INPUT_FILE)
# total_rows = len(df)
# print(f"   Loaded {total_rows:,} rows")
#
# # --- PROCESS ---
# print(f"\n⚙ Processing all rows...")
# print(f"   Extracting price + {CONTEXT_WORDS} words before/after...")
# tqdm.pandas(desc="Extracting prices")
# df["price_text"] = df[TEXT_COLUMN].progress_apply(extract_price_with_context)
#
# # --- SAVE ---
# print(f"\n Saving results...")
# df.to_csv(OUTPUT_FILE, index=False)
#
# # --- STATISTICS ---
# found_count = df["price_text"].notna().sum()
# not_found = total_rows - found_count
#
# print(f"\n COMPLETE!")
# print(f"   ✓ Found prices: {found_count:,} rows ({found_count/total_rows*100:.1f}%)")
# print(f"   ✗ No price found: {not_found:,} rows ({not_found/total_rows*100:.1f}%)")
# print(f"    Saved to: {OUTPUT_FILE}")
#
# # --- SAMPLE RESULTS ---
# print("\n Sample extractions (first 15 with prices):")
# samples = df[df["price_text"].notna()].head(15)
# for idx, row in samples.iterrows():
#     print(f"\n   Row {idx}:")
#     print(f"   → {row['price_text']}")
#
# # --- SHOW ROWS WITHOUT PRICES ---
# print("\n\n Sample rows where NO price was found (first 5):")
# no_price = df[df["price_text"].isna()].head(5)
# for idx, row in no_price.iterrows():
#     preview = row[TEXT_COLUMN][:100].replace('\n', ' ')
#     print(f"\n   Row {idx}: {preview}...")
#
# print("\n\n Tips:")
# print("   • 'None' means no per-unit price was found")

## Converting land_area to simply "acre", and leaving it as float

In [3]:
# Regional average prices dictionary (in USD)
regional_prices = {
    "город Ташкент": 1251.69,  # Price segment: 10
    "Самаркандская область": 753.03,  # Price segment: 6
    "Ташкентская область": 695.25,  # Price segment: 5
    "Навоийская область": 583.55,  # Price segment: 4
    "Наманганская область": 470.91,  # Price segment: 2
    "Джизакская область": 464.21,  # Price segment: 2
    "Сурхандарьинская область": 455.77,  # Price segment: 2
    "Бухарская область": 454.60,  # Price segment: 2
    "Ферганская область": 444.84,  # Price segment: 2
    "Кашкадарьинская область": 443.78,  # Price segment: 2
    "Сырдарьинская область": 436.73,  # Price segment: 2
    "Хорезмская область": 397.03,  # Price segment: 2
    "Республика Каракалпакстан": 390.10  # Price segment: 1
}

# df['price_per_m2'] = (df['price'] / df['land_area']).round(2)

In [4]:
# df["land_area"] = (
#     df["land_area"]
#     .astype(str)                             # ensure string type
#     .str.replace("соток", "", regex=False)   # remove the word
#     .str.replace(" ", "", regex=False)        # remove inner spaces (like in '1 000')
#     .str.strip()                              # trim spaces
#     .replace("", None)                        # replace empty strings
#     .astype(float)                            # convert to float
# )
# df['land_area'].head()

In [5]:
# df["price_per_area"] = df["price"] / df["land_area"]
# df['price_per_area'].value_counts()


In [6]:
# df['purpose'].value_counts()

## Main cleaning

In [7]:
orig = pd.read_csv('olx_land_sale_orig.csv')
df = orig.copy()
df.rename(columns={'id_': 'id', 'url_':'url', 'title_': 'title', 'description_': 'description', 'status_': 'status', 'business_': 'business', 'last_refresh_time_': 'last_refresh_time', 'time__': 'time', 'price': 'price_usd', 'price_converted_value': 'price_uzs'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 39 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              22000 non-null  int64  
 1   url                             22000 non-null  object 
 2   title                           22000 non-null  object 
 3   description                     22000 non-null  object 
 4   business                        22000 non-null  int64  
 5   user_name                       22000 non-null  object 
 6   category_id                     22000 non-null  int64  
 7   category_type                   22000 non-null  object 
 8   location_region_name            22000 non-null  object 
 9   location_city_name              22000 non-null  object 
 10  location_district_name          21996 non-null  object 
 11  last_refresh_time               22000 non-null  object 
 12  created_time_                   

C:\Users\Saidakbar\AppData\Local\Temp\ipykernel_19228\1811184001.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  orig = pd.read_csv('olx_land_sale_orig.csv')


In [8]:
df.head()

,id,url,title,description,business,user_name,category_id,category_type,location_region_name,location_city_name,...,price_uzs,price_converted_currency,price_previous_value,price_converted_previous_value,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,https://www.olx.uz/d/obyavlenie/kuruk-er-sotil...,Курук ер сотилади 4 соту свет газ Сув интернет...,Ассалому алейкум Назарбекда<br />\n тукимачи м...,0,998992332145,10,real_estate,Ташкентская область,Назарбек,...,2.718807e+08,UZS,NaN,NaN,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,https://www.olx.uz/d/obyavlenie/prodaetsya-uch...,"Продаётся участок в центре, 2.5 сотки,вдоль до...","ПРОДАЖА УЧАСТКА <br />\nРайон: Яккасарайский,<...",1,Nigina,10,real_estate,Ташкентская область,Ташкент,...,3.304886e+09,UZS,NaN,NaN,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,https://www.olx.uz/d/obyavlenie/uch-kahramonda...,Уч кахрамондан 4 км кирганда Гулистонда 2 ёки ...,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,Акбар Исмаилов 2111,10,real_estate,Ташкентская область,Ташкент,...,2.764168e+08,UZS,NaN,NaN,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,https://www.olx.uz/d/obyavlenie/kibray-tsentr-...,Кибрай центр Продается земельный участок под с...,Продаётся земельный участок под строительство...,1,Хамза,10,real_estate,Ташкентская область,Кибрай,...,2.821104e+08,UZS,NaN,NaN,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,https://www.olx.uz/d/obyavlenie/srochno-uy-uri...,СРОЧНО!!! уй қуриш учун ер сотилади.,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,Алишер,10,real_estate,Ташкентская область,Кибрай,...,5.801046e+08,UZS,NaN,NaN,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


In [9]:
df['offer_type_'].value_counts()

offer_type_
offer    22000
Name: count, dtype: int64

In [10]:
cols_to_drop = ['url', 'title', 'user_name', 'category_id', 'last_refresh_time', 'created_time_',
                'valid_to_time_', 'pushup_time_', 'status', 'map_lat', 'map_lon', 'offer_type_', 'price_previous_value',
                'price_converted_previous_value']
df.drop(cols_to_drop, axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         22000 non-null  int64  
 1   description                22000 non-null  object 
 2   business                   22000 non-null  int64  
 3   category_type              22000 non-null  object 
 4   location_region_name       22000 non-null  object 
 5   location_city_name         22000 non-null  object 
 6   location_district_name     21996 non-null  object 
 7   promotion_highlighted      22000 non-null  int64  
 8   promotion_urgent           22000 non-null  int64  
 9   promotion_top_ad           22000 non-null  int64  
 10  promotion_premium_ad_page  22000 non-null  int64  
 11  time                       22000 non-null  object 
 12  price_usd                  22000 non-null  float64
 13  price_currency             22000 non-null  obj

In [12]:
df['time'] = pd.to_datetime(df['time'], format='%d_%m_%Y_%H_%M_%S').dt.normalize()

In [13]:
df.head()

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,Ассалому алейкум Назарбекда<br />\n тукимачи м...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,"ПРОДАЖА УЧАСТКА <br />\nРайон: Яккасарайский,<...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


In [14]:
# Clean up of html leftovers
df['description'] = df['description'].str.replace(r'<br\s*/?>', '', regex=True)
df.head(5)

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,Ассалому алейкум Назарбекда\n тукимачи маххала...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,"ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n Ра...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


## Fill null values in the land_area, to catch its values from the description and fill it

In [15]:
# Clean extracted number: remove spaces, fix comma
def clean_number(val):
    if pd.isna(val):
        return val
    val = re.sub(r'\s+', '', str(val))  # remove all spaces
    val = val.replace(',', '.')         # standardize decimal
    return float(val)

# Use the improved pattern
num_pattern = r'\d{1,3}(?:\s\d{3})*(?:[.,]\d+)?'
pattern = rf'({num_pattern})\s*[сc][оo0][тtт]\w*'

mask = (
    df['land_area'].isnull() &
    df['description'].notnull() &
    df['description'].str.contains(pattern, regex=True, case=False)
)

extracted = df.loc[mask, 'description'].str.extract(pattern, flags=re.IGNORECASE)
df.loc[mask, 'land_area'] = extracted[0].apply(clean_number)
print(df[['land_area', 'description']].head(10))
print("Remaining nulls in land_area:", df['land_area'].isnull().sum())

C:\Users\Saidakbar\AppData\Local\Temp\ipykernel_19228\1181913159.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['description'].str.contains(pattern, regex=True, case=False)


     land_area                                        description
0      4 соток  Ассалому алейкум Назарбекда\n тукимачи маххала...
1   2.50 соток  ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n   Ра...
2     24 соток  Зудлик билан 2 ёки 4  сотих ер учаска сотилади...
3      7 соток  Продаётся  земельный участок под строительство...
4      6 соток  Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...
5     40 соток  Продаётся земельный участок под производство и...
6      8 соток  Продаётся земельный участок с кадастром \nКибр...
7    150 соток  Продается база с земельным участок \nПаркентск...
8  1 000 соток  Представляю  Вашему Вниманию  Инвестиционное п...
9    600 соток  - 6 соток 20/30 метр.\n- 2 кават, 6 хона, 4 са...
Remaining nulls in land_area: 7


## Test4 21k

In [16]:
# SOTIX_REGEX = re.compile(
#     r'''
#     # Match number (supports: 6, 2.5, 4,5, 1 200, 10.000)
#     (?P<number1>\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)
#     \s*
#     # Optional: range indicator (dash, "dan", "gacha", "–", etc.)
#     (?:
#         [-–—~]?\s*                     # dash or no dash
#         (?:dan\s+)?                    # optional "dan"
#         (?P<number2>\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)?
#         \s*
#     )?
#     # Flexible "sot" unit (covers ALL variants in your data)
#     [сc][оo0][тtт]                    # "sot" in Cyrillic/Latin/OCR
#     \s*
#     # Optional suffixes (ix, ok, ik, ux, их, ок, их, ихлик, etc.)
#     [a-zа-я]*[ixkгоуhъь]*             # covers sotix, sotok, sotik, sutox, sotux, sotig‘, sutuk, so’tix, soʻtik, etc.
#     ''',
#     re.IGNORECASE | re.VERBOSE
# )
#
#
# def extract_sotix_from_desc(desc):
#     if pd.isna(desc):
#         return None
#
#     match = SOTIX_REGEX.search(str(desc))
#     if not match:
#         return None
#
#     def clean_num(s):
#         return float(s.replace(' ', '').replace(',', '.'))
#
#     try:
#         low = clean_num(match.group('number1'))
#         high_str = match.group('number2')
#         if high_str:
#             high = clean_num(high_str)
#             return (low + high) / 2.0
#         else:
#             return low
#     except (ValueError, TypeError):
#         return None

In [17]:
# # Extract sotix from description
# df['extracted_land_area'] = df['description'].apply(extract_sotix_from_desc)
#
# # Fill ONLY nulls in land_area
# df['land_area'] = df['land_area'].fillna(df['extracted_land_area'])

## Test3 2k

In [18]:
# # Main regex pattern
# SOTIX_PATTERN = r'''
# (\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)    # Group 1: number (6, 1 200, 2.5, 4,5)
# \s*                                    # optional space
# [-–—]?\s*                              # optional dash (for ranges)
# (?:dan\s+)?                            # optional "dan" (as in "6 dan 8")
# (\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)?    # Group 2: second number (for ranges)
# \s*                                    # optional space
# [сc][оo0][тtт]                        # flexible "sot" (cyrillic/latin/zero)
# \s*[a-zа-я]*[ixkгоуh]*                # optional endings: ix, ok, ik, ux, их, ок
# '''
#
# # Compile with verbose and ignore case
# compiled_pattern = re.compile(SOTIX_PATTERN, re.IGNORECASE | re.VERBOSE)

In [19]:
# def extract_sotix(desc):
#     if pd.isna(desc):
#         return None
#
#     # Search for pattern
#     match = compiled_pattern.search(str(desc))
#     if not match:
#         return None
#
#     num1_str = match.group(1)
#     num2_str = match.group(2)
#
#     # Clean number: remove spaces, fix comma
#     def clean_num(s):
#         return float(s.replace(' ', '').replace(',', '.'))
#
#     try:
#         low = clean_num(num1_str)
#         if num2_str:  # It's a range like "2-4" or "6 dan 8"
#             high = clean_num(num2_str)
#             return (low + high) / 2.0
#         else:
#             return low
#     except (ValueError, TypeError):
#         return None

In [20]:
# # Create extracted column
# df['extracted_land_area'] = df['description'].apply(extract_sotix)
#
# # Fill ONLY nulls in land_area
# df['land_area'] = df['land_area'].fillna(df['extracted_land_area'])
#
# # Optional: drop helper column
# # df = df.drop(columns=['extracted_land_area'])

## Test2 1.5k

In [21]:
# # Pattern to match:
# # - plain numbers: "6 sotix"
# # - ranges: "2-4 sotix", "6 dan 8 gacha", "5-6sotok"
# range_pattern = r'(\d+(?:[.,]\d+)?)(?:\s*[-–—]\s*|\s+dan\s+)(\d+(?:[.,]\d+)?)\s*[сc][оo0][тtт]\s*[a-zа-я]*[ixkгоуh]*'
#
# # Fallback: single number (your original case)
# single_pattern = r'(\d+(?:[.,]\d+)?(?:\s*\d{3})*(?:[.,]\d+)?)\s*[сc][оo0][тtт]\s*[a-zа-я]*[ixkгоуh]*'
#
# def extract_sotix_robust(desc):
#     if pd.isna(desc):
#         return None
#
#     desc = str(desc)
#
#     # First: try to match a RANGE
#     range_match = re.search(range_pattern, desc, re.IGNORECASE)
#     if range_match:
#         try:
#             low = float(range_match.group(1).replace(',', '.'))
#             high = float(range_match.group(2).replace(',', '.'))
#             return (low + high) / 2.0
#         except ValueError:
#             pass
#
#     # Second: try single number (with space support: "1 000")
#     single_match = re.search(single_pattern, desc, re.IGNORECASE)
#     if single_match:
#         num_str = single_match.group(1).replace(' ', '').replace(',', '.')
#         try:
#             return float(num_str)
#         except ValueError:
#             pass
#
#     return None

In [22]:
# df['extracted_land_area'] = df['description'].apply(extract_sotix_robust)
#
# # Fill ONLY the NaNs in land_area
# df['land_area'] = df['land_area'].fillna(df['extracted_land_area'])

## Test1 500

In [23]:
# sotix_pattern = r'(\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)\s*[сc][оo0][тtт]\s*[а-яa-z]*\s*[\d\w]*'
#
# # Explanation:
# # - \d{1,3}(?:\s?\d{3})* → matches 6, 1 200, 1200, 10 000
# # - [.,]\d+? → accepts 6.5 or 6,5
# # - [сc][оo0][тtт] → flexible "с/с", "о/o/0", "т/t"
# # - \s*[а-яa-z]*\s*[\d\w]* → matches endings like "их", "ok", "ka", "ux", "ig", etc.
#
#
# def extract_sotix(text):
#     if pd.isna(text):
#         return None
#     match = re.search(sotix_pattern, text, re.IGNORECASE)
#     if match:
#         num_str = match.group(1)
#         # Clean: remove spaces, replace comma with dot
#         num_clean = num_str.replace(' ', '').replace(',', '.')
#         try:
#             return float(num_clean)
#         except ValueError:
#             return None
#     return None

In [24]:
# # Create new column: extracted_land_area
# df['extracted_land_area'] = df['description'].apply(extract_sotix)
#
# # Optional: fill land_area if null (as you planned)
# mask = df['land_area'].isnull() & df['extracted_land_area'].notnull()
# df.loc[mask, 'land_area'] = df['extracted_land_area']
#
# # Check results
# print(df[['land_area', 'extracted_land_area', 'description']].head(10))
# print("Remaining nulls in land_area:", df['land_area'].isnull().sum())

In [25]:
df['description'][8]

'Представляю  Вашему Вниманию  Инвестиционное предложение \n\nпод  застройку  любого типа недвижимости, ориентир Турбаза  "Золотой корабль. Рядом речка "Чирчик.\n\nЗемельный участок с кадастром \nВдоль дороги трасса Ташкент-Чарвак, Бостанлыкский район, 1 линия!!\n\nИмеется проект для зоны отдыха. \n\nИдеальное расположение  для:\nГостиницы \nЗоны отдыха \nАпарт отеля \nКоттеджа\nДачу \nСанатория\nКлиника\n1.05 гектар\nнебольшой торг  имеется !\n\nЦена - 735000у.е.\n\nТел:+99893-801-99-57 Рустам \nСпециалист  по  недвижимости.'

In [26]:
df.head()

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,Ассалому алейкум Назарбекда\n тукимачи маххала...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,"ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n Ра...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


In [27]:
df['land_area'].value_counts()

land_area
6 соток        2913
4 соток        1875
8 соток        1455
10 соток       1160
5 соток        1032
               ... 
6.84 соток        1
5.25 соток        1
16.74 соток       1
4.33 соток        1
288 соток         1
Name: count, Length: 867, dtype: int64

In [28]:
# df['extracted_land_area'].value_counts()

In [29]:
# df.info()
# df[['land_area', 'extracted_land_area']]

In [30]:
# df['land_area'].value_counts()
# df['extracted_land_area'].value_counts()

In [31]:
# df['log_land_area'] = np.log(df['extracted_land_area'])
#
# plt.figure(figsize=(8,5))
# plt.hist(df['log_land_area'], bins=20, edgecolor='black')
# plt.title('Distribution of Log-Transformed Land Area')
# plt.xlabel('log(Land Area)')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()

In [32]:
# # Filter rows where extracted_land_area is NaN and description is not null
# unmatched = df[
#     df['extracted_land_area'].isna() &
#     df['description'].notna()
# ]
#
# # Save *only* the 'description' column (full text, no index, no extra columns)
# unmatched['description'].to_csv(
#     'unmatched_descriptions_full.csv',
#     index=False,
#     header=False,      # ← no header, just raw text (easier for pattern mining)
#     encoding='utf-8'
# )

## Second step of pattern mining

In [33]:
# def extract_land_area(desc):
#     if pd.isna(desc):
#         return None
#     desc = str(desc)
#
#     # Try patterns in order: sotix → sqm/kv → m2 → dimensions → range
#     for pat_name, pat in [
#         ('sotix', r'(?P<val>\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)\s*[сc][оo0][тtт]\s*[а-яa-z]*[ixkгоуhъь]*'),
#         ('sqm',   r'(?P<val>\d{1,4}(?:\s?\d{3})*)\s*(?:кв\.?м?|кв|kv\.?m?|m2|sq\.?m?|квадрат|kvadrat|квт|kv\.t)'),
#         ('m2',    r'(?P<val>\d{1,4}(?:\s?\d{3})*)\s*m[²2]'),
#         ('dim',   r'(?P<w>\d{1,3})\s*[x×]\s*(?P<h>\d{1,3})\s*(?:m|metr|метр)?'),
#         ('range', r'(?P<low>\d+(?:[.,]\d+)?)\s*(?:[-–—~]|dan)\s*(?P<high>\d+(?:[.,]\d+)?)\s*[сc][оo0][тtт]'),
#     ]:
#         match = re.search(pat, desc, re.IGNORECASE)
#         if not match:
#             continue
#
#         try:
#             # Helper
#             def clean_num(s):
#                 return float(s.replace(' ', '').replace(',', '.'))
#
#             groups = match.groupdict()
#
#             if pat_name == 'sotix':
#                 return clean_num(groups['val'])
#
#             elif pat_name in ('sqm', 'm2'):
#                 return round(clean_num(groups['val']) / 100.0, 2)
#
#             elif pat_name == 'dim':
#                 w = clean_num(groups['w'])
#                 h = clean_num(groups['h'])
#                 return round((w * h) / 100.0, 2)
#
#             elif pat_name == 'range':
#                 low = clean_num(groups['low'])
#                 high = clean_num(groups['high'])
#                 return round((low + high) / 2.0, 2)
#
#         except (ValueError, TypeError, KeyError):
#             continue
#
#     return None

In [34]:
# # Extract from ALL descriptions (not just nulls — for debugging)
# df['extracted_land_area_full'] = df['description'].apply(extract_land_area)
#
# # Fill ONLY where land_area is NaN
# df['land_area'] = df['land_area'].fillna(df['extracted_land_area_full'])
#
# # Optional: see how many new fills
# new_fills = df['extracted_land_area'].notna().sum()
# print(f"✅ Filled {new_fills} additional rows with robust extractor")
# print("Remaining null land_area:", df['land_area'].isnull().sum())

In [35]:
df.head(10)

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,Ассалому алейкум Назарбекда\n тукимачи маххала...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,"ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n Ра...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый
5,50323632,Продаётся земельный участок под производство и...,1,real_estate,Ташкентская область,Ташкент,Мирзо-Улугбекский район,0,0,0,...,0,1,1.923480e+08,UZS,40 соток,Земля под строительство,"В городе, Вдоль трассы, В пригороде","Электричество, Газ, Вода, Канализация",Нет,Неделимый
6,54283143,Продаётся земельный участок с кадастром \nКибр...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,3.616278e+08,UZS,8 соток,Земля под строительство,В пригороде,"Электричество, Вода, Газ, Канализация, Интернет",Нет,NaN
7,55776194,Продается база с земельным участок \nПаркентск...,1,real_estate,Ташкентская область,Паркент,0,0,0,0,...,0,1,8.290490e+09,UZS,150 соток,Земля под строительство,"В городе, В пригороде",NaN,Нет,Неделимый
8,50900888,Представляю Вашему Вниманию Инвестиционное п...,1,real_estate,Ташкентская область,Искандар,0,0,0,0,...,0,1,9.342658e+09,UZS,1 000 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки",NaN,Нет,Неделимый
9,56741676,"- 6 соток 20/30 метр.\n- 2 кават, 6 хона, 4 са...",1,real_estate,Ташкентская область,Ташкент,Мирзо-Улугбекский район,0,0,0,...,0,1,5.791095e+09,UZS,600 соток,Земля под строительство,В городе,NaN,Да,NaN


In [36]:
# df['extracted_land_area_full'][500]

In [37]:
df['description'][500]

'Ховли сотилади. Умумий 7-сўтик. 2-хона времянка қуриб биткизилган. 4-хона фундаменти қурилган. Ховли айланасига ўралган. Сув бор. Свет бор. Газ бор. Кўча асфалт қилинган. Ховли боғча, мактаб ҳамда тўйхонага яқин. Ховлининг умумий суммаси 60$ минг доллар (келишилади). Кадастр хужжатлари мавжуд. Расмийлаштириш муаммосиз. Мурожаат учун тел: +998990603178  +998998557817 \nТошкент вилояти Юқоричирчиқ тумани Нурли йўл МФЙ Бархаёт кўчаси 2а уй.(Ориентир Жумабозор) (Рохат метродан 14-км)'

## 3rd layer of pattern mining


In [38]:
# # Optimized pattern — compact, supports comma/decimal/space/ranges/sqm/dim
# PATTERN = re.compile(
#     r'''
#     # 1. sotix: 6sotix, 8,5sotok, 7-сўтик, 6.5sutuk, 12soʻtik
#     (?P<sot>\d+(?:[.,\s]?\d*)?)\s*[сc][оoó0][тtт]\s*[a-zа-яʼʻ’\-]*[ixkгоуqх]*
#
#     |  # 2. sqm/kv/m² → /100: 600кв, 500 m2, 400sqm, 700м²
#     (?P<m2>\d+(?:\s?\d{3})*)\s*(?:кв\.?м?|кв|kv\.?m?|m²?|sq\.?m?|квт|квадрат)
#
#     |  # 3. dimensions → area/100: 20×30, 15/40, 10x25
#     (?P<w>\d{1,3})\s*[x×/]\s*(?P<h>\d{1,3})
#
#     |  # 4. ranges: 6-8, 6 dan 8, 6~8, 4-6sotok → avg
#     (?P<low>\d+(?:[.,]\d+)?)\s*[-–—~]|dan\s+(?P<high>\d+(?:[.,]\d+)?)
#     ''',
#     re.IGNORECASE | re.VERBOSE
# )
#
# def extract_sotix(desc):
#     if pd.isna(desc):
#         return None
#     desc = str(desc)
#
#     match = PATTERN.search(desc)
#     if not match:
#         return None
#
#     try:
#         def clean_num(val):
#             if val is None:
#                 return None
#             # Remove spaces, normalize comma → dot
#             return float(val.replace(' ', '').replace(',', '.'))
#
#         g = match.groupdict()
#
#         # sotix: "6sotix", "8,5 sotok", "7-сўтик"
#         if g['sot'] is not None:
#             return clean_num(g['sot'])
#
#         # sqm/kv/m² → convert to sotix
#         if g['m2'] is not None:
#             return round(clean_num(g['m2']) / 100.0, 2)
#
#         # dimensions → w × h → /100
#         if g['w'] is not None and g['h'] is not None:
#             w, h = clean_num(g['w']), clean_num(g['h'])
#             return round((w * h) / 100.0, 2)
#
#         # range: only process if BOTH low and high are present
#         if g['low'] is not None and g['high'] is not None:
#             low, high = clean_num(g['low']), clean_num(g['high'])
#             return round((low + high) / 2.0, 2)
#
#         # fallback: if only 'low' in range, return low (e.g., "6 dan" → assume 6)
#         if g['low'] is not None:
#             return clean_num(g['low'])
#
#     except (ValueError, TypeError):
#         pass
#
#     return None

In [39]:
# df['extracted_land_area'] = df['description'].apply(extract_sotix)
# df['land_area'] = df['land_area'].fillna(df['extracted_land_area'])
# print("Remaining nulls:", df['land_area'].isnull().sum())

## Pattern Mining for the hectars

In [40]:
# HECTARE_RE = re.compile(
#     r'(\d+(?:[.,]\d+)?)\s*(?:гектар[аы]?|га|gektar|hectare?|ha)\b',
#     re.IGNORECASE
# )
#
# def extract_hectare_to_sotix_if_missing(desc, current_extracted):
#     """
#     Only return hectare-based value if current_extracted is NaN/None.
#     Converts ha → sotix (×100), e.g., 1.05 гектар → 105.0
#     """
#     if pd.notna(current_extracted):
#         return current_extracted  # keep existing sotix/sqm value
#
#     if pd.isna(desc):
#         return current_extracted
#
#     match = HECTARE_RE.search(str(desc))
#     if match:
#         try:
#             ha = float(match.group(1).replace(',', '.'))
#             # Optional: filter implausible values (e.g., >50 ha)
#             if 0.01 <= ha <= 50.0:
#                 return round(ha * 100.0, 2)  # ha → sotix
#         except (ValueError, TypeError):
#             pass
#     return current_extracted

In [41]:
# # Update extracted_land_area: only fill where still NaN, using hectares
# df['extracted_land_area'] = df.apply(
#     lambda row: extract_hectare_to_sotix_if_missing(
#         row['description'],
#         row['extracted_land_area']
#     ),
#     axis=1
# )

In [42]:
# hectare_filled = (
#     df['extracted_land_area'].notna() &
#     df['description'].str.contains(r'гектар|га|gektar|ha\b', case=False, na=False)
# )
# print("✅ Filled via hectares:", hectare_filled.sum())

In [43]:
# df[['extracted_land_area', 'extracted_land_area_full']].head(10)

In [44]:
# df['extracted_land_area']

In [45]:
# sum(df['extracted_land_area'] >= 998)

## General checking

In [46]:
# #MAIN_PATTERN with expanded sotix and sq.m
# MAIN_PATTERN = re.compile(
#     r'''
#     # 1. SOTIX (expanded): 6sotix, 8,5sotok, 7-сўтик, 6.5sutuk, 12soʻtik, 4.2sotx, no space allowed
#     (?P<sot>
#         (?<!\d)\d+(?:[.,]\d+)?    # Number (not preceded by digit to avoid "600")
#         \s*                       # Optional space
#         [сc][оoóў0][тtт]          # Core: с/с, о/o/ó/ў/0, т/t (covers OCR, accents)
#         [a-zа-яʼʻ’\-]*           # Optional suffix: ix, ok, ik, ux, q, x, etc.
#         [ixkгоуqх]*              # Final suffix: sotix, sotok, sotq, etc.
#     )
#
#     |  # 1b. SOTIX (tight form): 6sotix, 12sotik, 4.2sotx (no space between number and unit)
#     (?P<sot_tight>
#         (?<!\d)\d+(?:[.,]\d+)?    # Number (not preceded by digit)
#         [сc][оoóў0][тtт]         # Core: с/с, о/o/ó/ў/0, т/t
#         [a-zа-яʼʻ’\-]*           # Optional suffix
#         [ixkгоуqх]*              # Final suffix
#     )
#
#     |  # 2. SQUARE METERS (expanded): 600кв, 500 m2, 400sqm, 700м², 200 квадрат, 600kv.m, etc.
#     (?P<sqm>
#         \d+(?:\s?\d{3})*         # Number: 600, 1 200, etc.
#         \s*                      # Optional space
#         (?:кв\.?м?|кв|kv\.?m?|m²?|sq\.?m?|квт|квадрат|kvadrat)\s* # Units (with optional .)
#         (?:метр|metr|м)?         # Optional "метр" or "metr"
#         (?:ли)?                  # Optional "ли" (e.g., "квадратли")
#     )
#
#     |  # 3. dimensions: 20×30, 15/40, 10x25 → area/100
#     (?P<w>\d{1,3})\s*[x×/]\s*(?P<h>\d{1,3})
#
#     |  # 4. ranges: 6-8, 6 dan 8, 4~6 sot → avg
#     (?P<low>\d+(?:[.,]\d+)?)\s*[-–—~]|dan\s+(?P<high>\d+(?:[.,]\d+)?)
#     ''',
#     re.IGNORECASE | re.VERBOSE
# )
#
# def extract_main_land_area_updated_v2(desc):
#     if pd.isna(desc):
#         return None
#     desc = str(desc)
#     match = MAIN_PATTERN.search(desc)
#     if not match:
#         return None
#
#     g = match.groupdict()
#     try:
#         clean = lambda s: float(s.replace(' ', '').replace(',', '.')) if s else None
#
#         sot_val = g.get('sot') or g.get('sot_tight')
#         if sot_val:
#             # Extract number from "6sotix", "8,5sotok", etc.
#             num_match = re.search(r'(?<!\d)\d+(?:[.,]\d+)?', sot_val)
#             if num_match:
#                 return clean(num_match.group(0))
#
#         elif g['sqm']:
#             # Extract number from "600кв", "200 квадрат метр", etc.
#             num_match = re.search(r'\d+(?:\s?\d{3})*', g['sqm'])
#             if num_match:
#                 return round(clean(num_match.group(0)) / 100.0, 2)
#
#         elif g['w'] and g['h']:
#             return round(clean(g['w']) * clean(g['h']) / 100.0, 2)
#
#         elif g['low'] and g['high']:
#             return round((clean(g['low']) + clean(g['high'])) / 2.0, 2)
#
#         elif g['low']:
#             return clean(g['low'])
#
#     except (ValueError, TypeError):
#         pass
#     return None

## For hectars

In [47]:
# MAIN_PATTERN = re.compile(
#     r'''
#     # 1. SOTIX (expanded): 6sotix, 8,5sotok, 7-сўтик, 6.5sutuk, 12soʻtik, 4.2sotx, no space allowed
#     (?P<sot>
#         (?<!\d)\d+(?:[.,]\d+)?    # Number (not preceded by digit to avoid "600")
#         \s*                       # Optional space
#         [сc][оoóў0][тtт]          # Core: с/с, о/o/ó/ў/0, т/t (covers OCR, accents)
#         [a-zа-яʼʻ’\-]*           # Optional suffix: ix, ok, ik, ux, q, x, etc.
#         [ixkгоуqх]*              # Final suffix: sotix, sotok, sotq, etc.
#     )
#
#     |  # 1b. SOTIX (tight form): 6sotix, 12sotik, 4.2sotx (no space between number and unit)
#     (?P<sot_tight>
#         (?<!\d)\d+(?:[.,]\d+)?    # Number (not preceded by digit)
#         [сc][оoóў0][тtт]         # Core: с/с, о/o/ó/ў/0, т/t
#         [a-zа-яʼʻ’\-]*           # Optional suffix
#         [ixkгоуqх]*              # Final suffix
#     )
#
#     |  # 2. SQUARE METERS (expanded): 600кв, 500 m2, 400sqm, 700м², 200 квадрат, 600kv.m, etc.
#     (?P<sqm>
#         \d+(?:\s?\d{3})*         # Number: 600, 1 200, etc.
#         \s*                      # Optional space
#         (?:кв\.?м?|кв|kv\.?m?|m²?|sq\.?m?|квт|квадрат|kvadrat)\s* # Units (with optional .)
#         (?:метр|metr|м)?         # Optional "метр" or "metr"
#         (?:ли)?                  # Optional "ли" (e.g., "квадратли")
#     )
#
#     |  # 3. HECTARES: 1.05 гектар, 0,6 га, 2 gektar, etc. (safe from phone numbers)
#     (?P<ha>
#         (?<!\d\s)                # Negative lookbehind: not after "number space"
#         \d+(?:[.,]\d+)?          # Number: 1, 1.05, 0,6, 2.3
#         \s*                      # Optional space
#         (?:                      # Unit group (non-capturing)
#             гектар[аы]?         # гектар, гектара
#             | га\b              # га (word boundary)
#             | [gG][eеё][kк][tт][aа]р?\b # gektar, гектар (OCR tolerant)
#             | hectare?\b        # hectare
#             | ha\b              # ha
#         )
#         (?!.*(?:\b\d{2,}\s*\d{2,})) # Negative lookahead: avoid phone-like numbers
#     )
#
#     |  # 4. dimensions: 20×30, 15/40, 10x25 → area/100
#     (?P<w>\d{1,3})\s*[x×/]\s*(?P<h>\d{1,3})
#
#     |  # 5. ranges: 6-8, 6 dan 8, 4~6 sot → avg
#     (?P<low>\d+(?:[.,]\d+)?)\s*[-–—~]|dan\s+(?P<high>\d+(?:[.,]\d+)?)
#     ''',
#     re.IGNORECASE | re.VERBOSE
# )
#
# def extract_main_land_area_with_hectares(desc):
#     if pd.isna(desc):
#         return None
#     desc = str(desc)
#     match = MAIN_PATTERN.search(desc)
#     if not match:
#         return None
#
#     g = match.groupdict()
#     try:
#         clean = lambda s: float(s.replace(' ', '').replace(',', '.')) if s else None
#
#         sot_val = g.get('sot') or g.get('sot_tight')
#         if sot_val:
#             # Extract number from "6sotix", "8,5sotok", etc.
#             num_match = re.search(r'(?<!\d)\d+(?:[.,]\d+)?', sot_val)
#             if num_match:
#                 return clean(num_match.group(0))
#
#         elif g['sqm']:
#             # Extract number from "600кв", "200 квадрат метр", etc.
#             num_match = re.search(r'\d+(?:\s?\d{3})*', g['sqm'])
#             if num_match:
#                 return round(clean(num_match.group(0)) / 100.0, 2)
#
#         elif g['ha']:
#             # Extract number from "1.05 гектар", "0,6 га", etc.
#             num_match = re.search(r'(?<!\d)\d+(?:[.,]\d+)?', g['ha'])
#             if num_match:
#                 ha = clean(num_match.group(0))
#                 # Filter implausible values "600 га"
#                 if 0.01 <= ha <= 50.0:
#                     return round(ha * 100.0, 2)  # ha → sotix
#
#         elif g['w'] and g['h']:
#             return round(clean(g['w']) * clean(g['h']) / 100.0, 2)
#
#         elif g['low'] and g['high']:
#             return round((clean(g['low']) + clean(g['high'])) / 2.0, 2)
#
#         elif g['low']:
#             return clean(g['low'])
#
#     except (ValueError, TypeError):
#         pass
#     return None

In [48]:
# # Apply the new function that includes hectare extraction
# df['extracted_land_area'] = df['description'].apply(extract_main_land_area_with_hectares)
#
# print("Filled by main extractor (including hectares):", df['extracted_land_area'].notna().sum())
# print("Remaining nulls:", df['extracted_land_area'].isnull().sum())

In [49]:
# df['extracted_land_area'] >=998

In [50]:
df.head()

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,Ассалому алейкум Назарбекда\n тукимачи маххала...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,"ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n Ра...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


In [51]:
# df['extracted_land_area']

In [52]:
# nonev = df[df['extracted_land_area'].isna()]
# nonev[['description', 'extracted_land_area']]


In [53]:
## Implementing pattern for remaining 8k nan's

In [54]:
# Updated MAIN_PATTERN with ALL your examples and file analysis
MAIN_PATTERN = re.compile(
    r'''
    # 1. SOTIX (expanded): 6sotix, 8,5sotok, 7-сўтик, 6.5sutuk, 12soʻtik, 4.2sotx, no space allowed
    (?P<sot>
        (?<!\d)\d+(?:[.,]\d+)?    # Number (not preceded by digit to avoid "600")
        \s*                       # Optional space
        [сc][оoóў0][тtт]          # Core: с/с, о/o/ó/ў/0, т/t (covers OCR, accents)
        [a-zа-яʼʻ’\-]*           # Optional suffix: ix, ok, ik, ux, q, x, etc.
        [ixkгоуqхл]*              # Final suffix: sotix, sotok, sotq, sotixli, etc.
    )

    |  # 1b. SOTIX (tight form): 6sotix, 12sotik, 4.2sotx (no space between number and unit)
    (?P<sot_tight>
        (?<!\d)\d+(?:[.,]\d+)?    # Number (not preceded by digit)
        [сc][оoóў0][тtт]         # Core: с/с, о/o/ó/ў/0, т/t
        [a-zа-яʼʻ’\-]*           # Optional suffix
        [ixkгоуqхл]*              # Final suffix: sotix, sotok, sotq, sotixli, etc.
    )

    |  # 1c. SOTIX ABBREVIATION: 3 ст, 2 ст (common abbreviation for "сот")
    (?P<sot_short>
        (?<!\d)\d+(?:[.,]\d+)?    # Number
        \s*                       # Optional space
        ст\b                      # "ст" abbreviation (word boundary to avoid "состав")
    )

    |  # 2. SQUARE METERS (expanded): 600кв, 500 m2, 400sqm, 700м², 200 квадрат, 600kv.m, etc.
    (?P<sqm>
        \d+(?:\s?\d{3})*         # Number: 600, 1 200, etc.
        \s*                      # Optional space
        (?:кв\.?м?|кв|kv\.?m?|m²?|sq\.?m?|квт|квадрат|kvadrat)\s* # Units (with optional .)
        (?:метр|metr|м)?         # Optional "метр" or "metr"
        (?:ли)?                  # Optional "ли" (e.g., "квадратли")
    )

    |  # 3. HECTARES: 1.05 гектар, 0,6 га, 2 gektar, etc. (safe from phone numbers)
    (?P<ha>
        (?<!\d\s)                # Negative lookbehind: not after "number space"
        \d+(?:[.,]\d+)?          # Number: 1, 1.05, 0,6, 2.3
        \s*                      # Optional space
        (?:                      # Unit group (non-capturing)
            гектар[аы]?         # гектар, гектара
            | га\b              # га (word boundary)
            | [gG][eеё][kк][tт][aа]р?\b # gektar, гектар (OCR tolerant)
            | hectare?\b        # hectare
            | ha\b              # ha
        )
        (?!.*(?:\b\d{2,}\s*\d{2,})) # Negative lookahead: avoid phone-like numbers
    )

    |  # 4. DIMENSIONS: 20*30, 20×30, 15/40, 10x25 → area/100
    (?P<w>\d{1,3})\s*[x×*]\s*(?P<h>\d{1,3})  # Added * to cover 20*30

    |  # 5. RANGES: 6-8, 6 dan 8, 4~6 sot → avg
    (?P<low>\d+(?:[.,]\d+)?)\s*[-–—~]|dan\s+(?P<high>\d+(?:[.,]\d+)?)
    ''',
    re.IGNORECASE | re.VERBOSE
)

def extract_main_land_area_with_hectares_updated(desc):
    if pd.isna(desc):
        return None
    desc = str(desc)
    match = MAIN_PATTERN.search(desc)
    if not match:
        return None

    g = match.groupdict()
    try:
        clean = lambda s: float(s.replace(' ', '').replace(',', '.')) if s else None

        # Handle sotix (spaced, tight, short forms)
        sot_val = g.get('sot') or g.get('sot_tight') or g.get('sot_short')
        if sot_val:
            # Extract number from "6sotix", "8,5sotok", "3 ст", etc.
            num_match = re.search(r'(?<!\d)\d+(?:[.,]\d+)?', sot_val)
            if num_match:
                return clean(num_match.group(0))

        elif g['sqm']:
            # Extract number from "600кв", "200 квадрат метр", etc.
            num_match = re.search(r'\d+(?:\s?\d{3})*', g['sqm'])
            if num_match:
                return round(clean(num_match.group(0)) / 100.0, 2)

        elif g['ha']:
            # Extract number from "1.05 гектар", "0,6 га", etc.
            num_match = re.search(r'(?<!\d)\d+(?:[.,]\d+)?', g['ha'])
            if num_match:
                ha = clean(num_match.group(0))
                # Filter implausible values (likely typos like "600 га")
                if 0.01 <= ha <= 50.0:
                    return round(ha * 100.0, 2)  # ha → sotix

        elif g['w'] and g['h']:
            return round(clean(g['w']) * clean(g['h']) / 100.0, 2)

        elif g['low'] and g['high']:
            return round((clean(g['low']) + clean(g['high'])) / 2.0, 2)

        elif g['low']:
            return clean(g['low'])

    except (ValueError, TypeError):
        pass
    return None

# Check initial null count
# initial_nulls = df['extracted_land_area'].isna().sum()
# print(f"Initial nulls in 'extracted_land_area': {initial_nulls}")

In [55]:
# Apply the updated function
df['extracted_land_area'] = df['description'].apply(extract_main_land_area_with_hectares_updated)

# Check nulls after applying updated extractor
nulls_after_update = df['extracted_land_area'].isna().sum()
print(f"Nulls after applying updated extractor: {nulls_after_update}")

Nulls after applying updated extractor: 8975


## Applying fuzzy

In [56]:
from difflib import SequenceMatcher

def fuzzy_match_sotix(desc):
    if pd.isna(desc):
        return None
    desc = str(desc)
    # Look for "number + typo word" that could be "sotix"
    matches = re.findall(r'(\d+(?:[.,]\d+)?)\s*(\w+)', desc)
    for num, word in matches:
        # Fuzzy match: "ст", "sut", "sutu", "sot", "sotq", etc. → "sotix"
        if (SequenceMatcher(None, word.lower(), "сotix").ratio() > 0.5 or
            SequenceMatcher(None, word.lower(), "sot").ratio() > 0.6 or
            word.lower() in ["ст", "с", "sut", "sot", "sutu", "sotq"]):
            try:
                return float(num.replace(',', '.'))
            except:
                pass
    return None


mask = df['extracted_land_area'].isna()
df.loc[mask, 'extracted_land_area'] = df.loc[mask, 'description'].apply(fuzzy_match_sotix)


nulls_after_fuzzy = df['extracted_land_area'].isna().sum()
print(f"Nulls after fuzzy matching: {nulls_after_fuzzy}")
print(f"Filled by fuzzy matching: {nulls_after_update - nulls_after_fuzzy}")

Nulls after fuzzy matching: 5156
Filled by fuzzy matching: 3819


## Final Nan's filled and info

In [57]:

final_nulls = df['extracted_land_area'].isna().sum()
total_filled = nulls_after_update - final_nulls

print(f"\n=== FINAL SUMMARY ===")
print(f"Initial nulls: {nulls_after_update}")
print(f"Total filled: {total_filled}")
print(f"Remaining nulls: {final_nulls}")
print(f"Coverage: {((len(df) - final_nulls) / len(df) * 100):.2f}%")


=== FINAL SUMMARY ===
Initial nulls: 8975
Total filled: 3819
Remaining nulls: 5156
Coverage: 76.56%


In [58]:
ext_nulls = df[df['extracted_land_area'].isna()]
ext_nulls[['extracted_land_area', 'description']]

,extracted_land_area,description
28,NaN,Дачний участка зангатинскийрайон катартал буш ...
32,NaN,Bostonliq tumanida yer sotiladi tog' yonbag'ri...
40,NaN,Yuqori chirchiq tumani qipchoq maxalasi. Yer y...
53,NaN,Uy ozimi nomimdan suv gaz svet bor \nBoxca Ma...
61,NaN,Еэр сотилади 3 хонаси бор тувалет. газ су св...
...,...,...
21990,NaN,Урганч район бегавот кишлогида.Жой урнилари со...
21991,NaN,Жой урни сотилади.срочно.углавой да жойлашган....
21992,NaN,Uch xo'vizdo joylashgan 500 kivodirot turar j...
21993,NaN,"Урганч,хиваёл гайбуда 450 метр квадрат жой урн..."


In [59]:
# nan_desc_only = df[df['extracted_land_area'].isna()]
# descriptions_for_review = nan_desc_only[['description']].copy()
#
# descriptions_for_review.to_csv('descriptions_with_nan_land_area.csv', index=False, encoding='utf-8')

## Covering land area in text

In [60]:
import re
import pandas as pd

# Updated standalone pattern for fractional numbers in words ( "yarim" = half)
# More flexible to catch variations like "3 yarim", "1 yarmi", "yarim sotix", etc.
FRACTIONAL_WORDS_PATTERN = re.compile(
    r'''
    (?<!\d)                        # Negative lookbehind: not after a digit
    (?P<whole>\d+)?\s*             # Optional whole number part (e.g., "3" in "3 yarim")
    (?:\s*(?:[+]\s*)?              # Optional plus sign
    (?:yarim|yarmi|yarım|yarmı))   # Match variations of "yarim" (half)
    \s*                            # Optional space
    [сc][оoóў0][тtт]               # Core sotix unit (с/с, о/o/ó/ў/0, т/t)
    [a-zа-яʼʻ’\-]*                # Optional suffix: ix, ok, ik, etc.
    [ixkгоуqхл]*                   # Final suffix: sotix, sotok, etc.
    ''',
    re.IGNORECASE | re.VERBOSE
)

def extract_fractional_words_only_v2(desc):
    """
    Extract land area from text like "3 yarim sotix", "5 yarmi sotik", "yarim sotix".
    Only applies if the description contains these specific word forms.
    """
    if pd.isna(desc):
        return None
    desc = str(desc)

    match = FRACTIONAL_WORDS_PATTERN.search(desc)
    if match:
        try:
            whole_part = match.group('whole')
            if whole_part:  # If there's a whole number ( "3 yarim")
                whole_num = int(whole_part)
                return float(whole_num) + 0.5  # Add 0.5 for "yarim"/"yarmi"
            else:  # If there's no whole number ( "yarim sotix")
                return 0.5  # Just "yarim" = 0.5
        except (ValueError, TypeError):
            pass
    return None

In [61]:

mask = df['extracted_land_area'].isna()
extracted_values = df.loc[mask, 'description'].apply(extract_fractional_words_only_v2)

newly_filled = extracted_values.notna().sum()
print(f"Filled by fractional words pattern: {newly_filled}")
df.loc[mask, 'extracted_land_area'] = extracted_values

Filled by fractional words pattern: 0


C:\Users\Saidakbar\AppData\Local\Temp\ipykernel_19228\2007062000.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[None None None ... None None None]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'extracted_land_area'] = extracted_values


## Addressing missing 5k values


In [62]:
# Pattern for missed sotix variations
SOTIX_EXTRA_PATTERN = re.compile(
    r'''
    (?<!\d)                        # Negative lookbehind: not after a digit
    (?P<whole>\d+)?\s*             # Optional whole number part (e.g., "3" in "3 yarim")
    (?:\s*(?:[+]\s*)?              # Optional plus sign
    (?:yarim|yarmi|yarım|yarmı))?  # Optional fractional part (e.g., "yarim")
    \s*                            # Optional space
    (?:                            # Group of number words (e.g., "uch yarim")
        (?:bir|ikki|uch|tort|besh|olti|yetti|sakkiz|to'qqiz|o'n)\s+
        (?:yarim|yarmi|yarım|yarmı)
        |
        (?:bir|ikki|uch|tort|besh|olti|yetti|sakkiz|to'qqiz|o'n)
    )?
    \s*                            # Optional space
    [сc][оoóў0][тtт]               # Core sotix unit (с/с, о/o/ó/ў/0, т/t)
    [a-zа-яʼʻ’\-]*                # Optional suffix: ix, ok, ik, etc.
    [ixkгоуqхл]*                   # Final suffix: sotix, sotok, etc.
    ''',
    re.IGNORECASE | re.VERBOSE
)

def extract_sotix_extra_only(desc):
    if pd.isna(desc):
        return None
    desc = str(desc)

    match = SOTIX_EXTRA_PATTERN.search(desc)
    if match:
        try:
            whole_part = match.group('whole')
            matched_text = match.group(0)

            # Check for fractional part like "yarim"
            has_half = any(word in matched_text.lower() for word in ['yarim', 'yarmi', 'yarım', 'yarmı'])

            if whole_part:
                whole_num = int(whole_part)
                if has_half:
                    return float(whole_num) + 0.5
                else:
                    return float(whole_num)
            else:
                # Check for number words like "uch yarim", "ikki"
                text = match.group(0).lower()
                number_map = {
                    'bir': 1, 'ikki': 2, 'uch': 3, 'tort': 4, 'besh': 5,
                    'olti': 6, 'yetti': 7, 'sakkiz': 8, 'to\'qqiz': 9, 'o\'n': 10
                }
                for word, num in number_map.items():
                    if word in text:
                        if has_half:
                            return float(num) + 0.5
                        else:
                            return float(num)
        except (ValueError, TypeError):
            pass
    return None

initial_nulls = df['extracted_land_area'].isna().sum()
print(f"Initial nulls: {initial_nulls}")


mask = df['extracted_land_area'].isna()
df.loc[mask, 'extracted_land_area'] = df.loc[mask, 'description'].apply(extract_sotix_extra_only)

nulls_after_sotix = df['extracted_land_area'].isna().sum()
print(f"Nulls after sotix extra pattern: {nulls_after_sotix}")
print(f"Filled by sotix extra pattern: {initial_nulls - nulls_after_sotix}")

Initial nulls: 5156
Nulls after sotix extra pattern: 5154
Filled by sotix extra pattern: 2


In [63]:
# Pattern for missed sq.m variations
SQM_EXTRA_PATTERN = re.compile(
    r'''
    (?P<value>\d+(?:\s?\d{3})*\s*)  # Number: 600, 1 200, etc.
    (?:                            # Units (with optional .)
        квадрат|kvadrat            # "квадрат", "kvadrat"
        | кв\.?м?|кв|kv\.?m?       # "кв", "kv.m", etc.
        | m²?|m2                   # "m²", "m2"
        | sq\.?m?                  # "sq.m"
        | квадратметр|kvadratmetr  # "квадратметр"
    )
    (?:метр|metr|м)?               # Optional "метр" or "metr"
    (?:ли)?                        # Optional "ли" (e.g., "квадратли")
    ''',
    re.IGNORECASE | re.VERBOSE
)

def extract_sqm_extra_only(desc):
    if pd.isna(desc):
        return None
    desc = str(desc)

    match = SQM_EXTRA_PATTERN.search(desc)
    if match:
        try:
            val = float(match.group('value').replace(' ', ''))
            return round(val / 100.0, 2)  # Convert to sotix
        except (ValueError, TypeError):
            pass
    return None

mask = df['extracted_land_area'].isna()
df.loc[mask, 'extracted_land_area'] = df.loc[mask, 'description'].apply(extract_sqm_extra_only)

nulls_after_sqm = df['extracted_land_area'].isna().sum()
print(f"Nulls after sq.m extra pattern: {nulls_after_sqm}")
print(f"Filled by sq.m extra pattern: {nulls_after_sotix - nulls_after_sqm}")

Nulls after sq.m extra pattern: 5048
Filled by sq.m extra pattern: 106


In [64]:
# Pattern for missed dimension variations
DIM_EXTRA_PATTERN = re.compile(
    r'''
    (?P<w>\d+(?:[.,]\d+)?)\s*      # Width: 20
    [x×*]\s*                       # Separator: x, ×, *
    (?P<h>\d+(?:[.,]\d+)?)         # Height: 30
    (?:\s*m)?                      # Optional "m"
    ''',
    re.IGNORECASE | re.VERBOSE
)

def extract_dim_extra_only(desc):
    if pd.isna(desc):
        return None
    desc = str(desc)

    match = DIM_EXTRA_PATTERN.search(desc)
    if match:
        try:
            w = float(match.group('w').replace(',', '.'))
            h = float(match.group('h').replace(',', '.'))
            return round((w * h) / 100.0, 2)  # Convert to sotix
        except (ValueError, TypeError):
            pass
    return None


mask = df['extracted_land_area'].isna()
df.loc[mask, 'extracted_land_area'] = df.loc[mask, 'description'].apply(extract_dim_extra_only)

nulls_after_dim = df['extracted_land_area'].isna().sum()
print(f"Nulls after dimension extra pattern: {nulls_after_dim}")
print(f"Filled by dimension extra pattern: {nulls_after_sqm - nulls_after_dim}")

Nulls after dimension extra pattern: 5045
Filled by dimension extra pattern: 3


In [65]:
# Pattern for missed hectare variations
HA_EXTRA_PATTERN = re.compile(
    r'''
    (?<!\d\s)                      # Negative lookbehind: not after "number space"
    (?P<value>\d+(?:[.,]\d+)?)     # Number: 1, 1.05, 0,6, 2.3
    \s*                            # Optional space
    (?:                            # Unit group (non-capturing)
        гектар[аы]?               # гектар, гектара
        | га\b                    # га (word boundary)
        | [gG][eеё][kк][tт][aа]р?\b # gektar, гектар (OCR tolerant)
        | hectare?\b              # hectare
        | ha\b                    # ha
    )
    (?!.*(?:\b\d{2,}\s*\d{2,}))    # Negative lookahead: avoid phone-like numbers
    ''',
    re.IGNORECASE | re.VERBOSE
)

def extract_ha_extra_only(desc):
    if pd.isna(desc):
        return None
    match = HA_EXTRA_PATTERN.search(str(desc))
    if match:
        try:
            ha = float(match.group('value').replace(',', '.'))
            if 0.01 <= ha <= 50.0:
                return round(ha * 100.0, 2)  # ha → sotix
        except (ValueError, TypeError):
            pass
    return None

mask = df['extracted_land_area'].isna()
df.loc[mask, 'extracted_land_area'] = df.loc[mask, 'description'].apply(extract_ha_extra_only)

nulls_after_ha = df['extracted_land_area'].isna().sum()
print(f"Nulls after hectare extra pattern: {nulls_after_ha}")
print(f"Filled by hectare extra pattern: {nulls_after_dim - nulls_after_ha}")

Nulls after hectare extra pattern: 5045
Filled by hectare extra pattern: 0


In [66]:
# Final summary
final_nulls = df['extracted_land_area'].isna().sum()
total_filled = initial_nulls - final_nulls

print(f"\n=== FINAL SUMMARY ===")
print(f"Initial nulls: {initial_nulls}")
print(f"Total filled by extra patterns: {total_filled}")
print(f"Remaining nulls: {final_nulls}")
print(f"Coverage: {((len(df) - final_nulls) / len(df) * 100):.2f}%")


=== FINAL SUMMARY ===
Initial nulls: 5156
Total filled by extra patterns: 111
Remaining nulls: 5045
Coverage: 77.07%


In [72]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)   # optional
pd.set_option('display.max_columns', None)



In [74]:
df

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,promotion_premium_ad_page,time,price_usd,price_currency,price_arranged,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot,extracted_land_area
0,52891158,Ассалому алейкум Назарбекда\n тукимачи маххалада 4 соту куру ер сотилади нархи 5000 мингдан 20 000 минг. Свет газ Сув хаммаси бор Мурожат учун тел +998992332145,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,0,2025-02-17,21000.0,UYE,1,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN,4.0
1,51622740,"ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n Ракат боши\nПлощадь: 2.5 сот\nФасад: 10 м\nМатериал: Кирпич.\n\nДополнительно:\n1 линия, вдоль дороги, удобный заезд , есть дополнительное место перед домом для парковки на .\n \n Цена: 260000 у.е\n\nПодробнее по телефону.\n +998956704848 \n +998956708383",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,1,2024-05-14,260000.0,UYE,1,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый,2.5
2,48788994,"Зудлик билан 2 ёки 4 сотих ер учаска сотилади, Юнусобод тумани (шахарга якин)га якин, 4 км, учаскани дакуметлари бор, учаскада сув, газ, свет, интернет, канализация бор, бурчакда жойлашган, ку́ча асфалт килинган реални кизиканла жойида гаплашамиз.",0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,0,2024-04-30,22000.0,UYE,0,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода, Интернет, Отопление",Нет,Делимый,4.0
3,56466143,Продаётся земельный участок под строительство \nКибрайский район \nКибрай цент \nОрентир: Водаканал\n7-соток \nТорец\nСтартовая цена 22 000 за сотку \nПодробности по телефону \n+998909120379\n+998886666888,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,1,2024-12-16,22000.0,UYE,1,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый,7.0
4,55098422,"Қибрай тумани Ёшлик маҳалласида жуда зўр жойдан зудлик билан ер сотилади тинч махалла, 2 та уй қурса болади. Локацияси яхши жойлашган катта йўлга якин, газ сув свет барча комуникациялар мавжуд уй қуруш учун манзаралий жойда жойлашган. Жойнинг умумий майдони 8 сотих ҳужжат бўйича 6 сотих, 2 сотих захват. Нарихидан келиштириб берилади.\nТел: 90 954 12 27",0,real_estate,Ташкентская область,Кибрай,0,0,0,0,0,2024-12-02,44999.0,UYE,1,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном массиве","Электричество, Вода, Газ",Нет,Делимый,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,56090518,Urganch tumani gʻallab mahallasi hayvat qishlogʻi Joy oʻrin sotiladi 600 kvadrat murojat uchun tel 880091005 narxiga keladon bolsak 30 million oxrgi baxosi,0,real_estate,Хорезмская область,Караул,0.0,0,0,0,0,2024-10-21,30000000.0,UZS,0,0,1,NaN,NaN,600 соток,Земля под строительство,В сельской местности,"Электричество, Телефон, Канализация, Вода, Интернет",Нет,Неделимый,6.0
21996,53101064,Урганч шахар санотчилар кучасида 1100кв цемент склад зудлик билан сотилади,0,real_estate,Хорезмская область,Ургенч,0.0,1,0,1,0,2024-07-22,85000.0,UYE,0,0,1,1.070116e+09,UZS,1 100 соток,Другое,В городе,"Вода, Электричество, Газ",Нет,Неделимый,11.0
21997,54688508,"Гулистон махалласи Дурдона кучаси жасмин магазинни орка тарафида,200 кв кумилган фундамент килинган турар жой урни,хусусийлаштирилган,нархи 5000$ бартерга машинахам булоди,озгина келишамиз",0,real_estate,Хорезмская область,Хива,0.0,0,0,0,0,2024-06-24,5000.0,UYE,1,0,1,6.306400e+07,UZS,200 соток,Земля под строительство,В городе,NaN,Да,NaN,2.0
21998,52119884,Срочно сотилоди бохосини\n галиштирип

In [ ]:
df[['description','price_usd','land_area']].loc[15:22]

,description,price_usd,land_area
15,"Продаётся земельный участок под строительство \nКибрайский район Дурмень \nОрентир: Саниф, Новая махалля \nФасад 15×20\n3-соток \nСтартовая цена 30 000\nПодробности по телефону \n+998909120379\n+998886666888",30000.0,3 соток
16,"ПРОДАЖА ЗЕМЛИ\nПОД СТРОИТЕЛЬСТВО ЖК\nСЕРГЕЛИ ХАНАБАД\nОРИЕНТИР БЫВШИЙ \nЖОКЕЙ КЛУБ \n\nПервая линия \n3 гектара\nГотовый с документами \nПод ЖК. Разрешение, проект есть\nрешение на 12 этажку\n100.000м2 \nИнфо по телефону \n\nЦена 33,500 за сотку \n\n903451234а",33500.0,300 соток
17,"Продается земельный участок под строительство.\nАдрес: Кибрайский район Кирпичный \n6-соток, \nземельный участок под строительство \nтерритория квадратная , \nесть временное дом. \nСтартовая цена 80 000 \nПодробности по телефону +998998330039 \n\nhttps://t.me/uchastok_uz\n\nhttps://www.instagram.com/invites/contact/?i=1wlvn5xmxlo6j&utm_content=ob30khg",80000.0,6 соток
18,"Продаётся земельный участок 8 соток, без кадастра, но с разрешением хокимията. Без стройки, но с ограждением стены. Находится в Кибрайском районе, махалля Маданият, за постом Уч Кахрамона. \n***\nЕр участкаси 8 сотих сотилади. Ер участкаси девор билан ўралган бўлиб кадастр қилинмаган. Манзил: Қибрай тумани Маданият махалла, постга Уч Кахрамон.",80000.0,8 соток
19,Срочно продаётся земля \nРабочий Гарадок 5сот земля вдоль дороги углавой 1линия под нежилой под бизнес центр под клиника,700000.0,5 соток
20,Продается коробка 3 сотки \n10/30 размер участка \n6 комнат \nВся комуникация свет газ канализация есть\n100.000$\nЕсть +можно поменяться на квартиру,100000.0,3 соток
21,Юқори Чирчиқ тумани Уртасаройда уй жой қуриш учун 3 сотих ер участка сотилади сотихи 5500$ оладиган одам тел қилсин бекорчилар безовта қилмасин газ свет сув бор води торассасини ёқасида +998946292760\n+998991218558,5500.0,300 соток
22,"3 сотих Ер участкаси сотилади. Продается земельный участок. 3 сот. 30 000$. Ориентир: таможенный пост Черняевка. Адрес: Ташкентская область, Ташкентский район, Махалля Сиргали, Улица Сулола, 65-дом. Тел.: +998917836100 +998950139990 +998977094363 Бекзод.",380000000.0,3 соток


In [84]:
# 2,21995,2199,11,21,22

df.loc[[2,21995,2199,11,21,22]][['description','price_usd','land_area']]

,description,price_usd,land_area
2,"Зудлик билан 2 ёки 4 сотих ер учаска сотилади, Юнусобод тумани (шахарга якин)га якин, 4 км, учаскани дакуметлари бор, учаскада сув, газ, свет, интернет, канализация бор, бурчакда жойлашган, ку́ча асфалт килинган реални кизиканла жойида гаплашамиз.",22000.0,24 соток
21995,Urganch tumani gʻallab mahallasi hayvat qishlogʻi Joy oʻrin sotiladi 600 kvadrat murojat uchun tel 880091005 narxiga keladon bolsak 30 million oxrgi baxosi,30000000.0,600 соток
2199,"Зангиота зиёротгохига шахардан кетишда Эркин постидан 400метр утиб Сергилига утиш йулида.\nКадастр хужжатлари бор.\nНархи:15500$ сотихи, келишилади.\n+998337920205",15500.0,7.60 соток
11,"Учкахрамондан 5 км отканда, Ункоргон мол бозор тарафда 8 сотик сотиги 20 000 $ дан\nВ 5ти км от Учкахрамана возле рынка Ункорган продается земля 8 соток, 1 сотка 20000$",20000.0,8 соток
21,Юқори Чирчиқ тумани Уртасаройда уй жой қуриш учун 3 сотих ер участка сотилади сотихи 5500$ оладиган одам тел қилсин бекорчилар безовта қилмасин газ свет сув бор води торассасини ёқасида +998946292760\n+998991218558,5500.0,300 соток
22,"3 сотих Ер участкаси сотилади. Продается земельный участок. 3 сот. 30 000$. Ориентир: таможенный пост Черняевка. Адрес: Ташкентская область, Ташкентский район, Махалля Сиргали, Улица Сулола, 65-дом. Тел.: +998917836100 +998950139990 +998977094363 Бекзод.",380000000.0,3 соток
